In [10]:
from utils import load_config

db = load_config("/root/ysl_project_clean/ysl_config.json")['db_connection']
print(db)

{'user': 'kering_admin', 'password': '#cow8TUEnbQ', 'host': 'pgm-uf6q0m0aa2zqx3z4co.pg.rds.aliyuncs.com', 'port': 5432, 'dbname': 'content_tagging', 'search_path': 'public'}


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from utils import write_to_postgres, load_config

# ======================================
# 1️⃣ 读取配置并建立连接
# ======================================
cfg = load_config("/root/ysl_project_clean/ysl_config.json")["db_connection"]

url = URL.create(
    drivername="postgresql+psycopg2",
    username=cfg["user"],
    password=cfg["password"],   # 支持 #、@、% 等特殊符号
    host=cfg["host"],
    port=int(cfg["port"]),
    database=cfg["dbname"]
)

engine = create_engine(url, pool_pre_ping=True, future=True)

df = pd.read_csv("/root/ysl_project_clean/Data_Cleaning/cleaned_df_v2.csv")

print(f"📦 成功加载 DataFrame: {df.shape[0]} 行 × {df.shape[1]} 列")

schema = "public"
table_name = "cleaned_post_v2"

print(f"🚀 正在将数据写入 {schema}.{table_name} ...")

write_to_postgres(
    df=df,
    table_name=f"{schema}.{table_name}",
    mode="overwrite",   # 可改为 append
    batch_size=10000
)

print(f"✅ 成功导出 {len(df)} 行到 {schema}.{table_name}")

# ======================================
# 4️⃣ 验证写入结果
# ======================================
with engine.connect() as conn:
    result = conn.execute(text(f'SELECT COUNT(*) FROM {schema}."{table_name}"')).scalar()
    print(f"📊 数据库中共有 {result} 行记录。")

🚀 正在将 DataFrame 写入 public.test_export ...
✅ 成功写入 3 行到表 public.test_export
📊 数据库中共有 3 行记录。
